In [1]:
from civicpy import civic

In [59]:
evidence = civic.get_all_evidence(include_status=['submitted', 'accepted'], allow_cached=False)

In [74]:
e.assertions

[<CIViC assertion 50>,
 <CIViC assertion 81>,
 <CIViC assertion 82>,
 <CIViC assertion 83>,
 <CIViC assertion 84>,
 <CIViC assertion 104>]

In [86]:
NUM_RETRIES = 10
def get_assertions(e):
    for i in range(0, NUM_RETRIES):
        try:
            return e.assertions
        except AttributeError as ex:
            pass
    return []

In [85]:
all_assertions = []
i = 0
for e in tqdm(evidence):
    try:
        #pass
        all_assertions.extend(get_assertions(e))
    except AttributeError as ex:
        print(e.name, ex)
        break
        i+=1
print(i / len(evidence))

  0%|          | 0/10603 [00:00<?, ?it/s]

EID3758 Expected evidence_items attribute for assertion, none found. 0
EID6940 Expected evidence_items attribute for assertion, none found. 0
EID7283 Expected evidence_items attribute for assertion, none found. 0
EID7319 Expected evidence_items attribute for assertion, none found. 0
EID8510 Expected evidence_items attribute for assertion, none found. 0
EID10800 Expected evidence_items attribute for assertion, none found. 0
EID10804 Expected evidence_items attribute for assertion, none found. 0
EID11109 Expected evidence_items attribute for assertion, none found. 0
EID11109 Expected evidence_items attribute for assertion, none found. 1


KeyboardInterrupt: 

In [50]:
import integration.orm.civic as orm
#from integration.parsers import Parser

In [51]:
def parse_evidence(
    evidence: civic.Evidence,
    parsed_sources,
) -> orm.Evidence:
    """Parse a civic Evidence object to its ORM representation."""
    #has_attached_disease = isinstance(evidence.disease, civic.Disease)
    #if evidence.source.id in parsed_sources.keys():
    #    source = parsed_sources[evidence.source.id]
    #else:
    #    source = parse_source(evidence.source)
    #   parsed_sources[evidence.source.id] = source
    return orm.Evidence(
        #description=evidence.description,
        #name=evidence.name,
        #direction=evidence.evidence_direction,
        assertions=[parse_assertion(a) for a in evidence.assertions],
        #level=evidence.evidence_level,
        #type_=evidence.evidence_type,
        #rating=evidence.rating,
        #significance=evidence.significance,
        #status=evidence.status,
        #disease_name=evidence.disease.name if has_attached_disease else None,
        #disease_display_name=evidence.disease.display_name
        #if has_attached_disease
        #else None,
        #disease_do_id=int(evidence.disease.doid)
        #if has_attached_disease and evidence.disease.doid
        #else None,
        #disease_do_id_url=evidence.disease.disease_url
        #if has_attached_disease
        #else None,
        #disease_cui=normalizer.do_id_to_cui(evidence.disease.doid)
        #if has_attached_disease
        #else None,
        #source=source,
        #molecular_profile_id=int(evidence.molecular_profile_id),
        #phenotypes=[parse_phenotype(p, normalizer) for p in evidence.phenotypes],
        #therapies=[parse_therapy(t, normalizer) for t in evidence.therapies],
    )

In [52]:
def parse_assertion(assertion: civic.Assertion) -> orm.Assertion:
    return None
    """Parse a civic Assertion object to its ORM representation."""
    return orm.Assertion(
        name=assertion.name,
        assertion_type=assertion.assertion_type,
        assertion_direction=assertion.assertion_direction,
        molecular_profile_id=int(assertion.molecular_profile_id),
        description=assertion.description,
        summary=assertion.summary,
        status=assertion.status,
        variant_origin=assertion.variant_origin,
        significance=assertion.significance,
        nccn_guideline_version=assertion.nccn_guideline_version,
        nccn_guideline_name=assertion.nccn_guideline.get("name")
        if isinstance(assertion.nccn_guideline, dict)
        else None,
        fda_regulatory_approval=assertion.fda_regulatory_approval,
        fda_companion_test=assertion.fda_companion_test,
        amp_level=assertion.amp_level,
    )


In [53]:
def parse_source(source: civic.Source) -> orm.Source:
    """Parse a civic Source object to its ORM representation."""
    return orm.Source(
        id=source.id,
        name=source.name,
        title=source.title,
        abstract=source.abstract,
        citation=source.citation,
        citation_id=source.citation_id,
        source_type=source.source_type,
        asco_abstract_id=source.asco_abstract_id,
        author_string=source.author_string,
        journal=source.full_journal_title,
        #date_publication=parse_publication_date(source.publication_date),
        pmc_id=source.pmc_id,
        #pm_id=get_pm_id_from_source_url(source.source_url),
        source_url=source.source_url,
        #clinical_trials=[parse_clinical_trial(t) for t in source.clinical_trials],
        #flags=orm.Flags(),
    )

In [54]:
from tqdm.auto import tqdm
for e in tqdm(evidence):
    parse_evidence(e, {})

  0%|          | 0/10939 [00:00<?, ?it/s]

AttributeError: Expected evidence_items attribute for assertion, none found.

In [14]:
from sqlalchemy import select

In [15]:
from api.app import app
from fastapi.testclient import TestClient
import os

with TestClient(app) as client:
    # Initialize state
    pass    

session = app.state.session()

# GGPONC

In [16]:
from integration.orm import ggponc

In [17]:
len(session.scalars(select(ggponc.Guideline)).fetchall())

34

In [18]:
import pandas as pd

def top_10(cuis):
    s = pd.Series(cuis).value_counts()[0:10]
    df = pd.DataFrame(s)
    df['name'] = df.index.map(app.state.concept_parser.umls_parser.get_umls_text)
    return df

def assert_not_null(cuis):
    assert all([c != None for c in cuis])

In [19]:
top_pop = session.scalars(select(ggponc.Population.cui)).fetchall()
all_pops = session.scalars(select(ggponc.SubPopulation.cui)).fetchall()
ggponc_pops = top_pop + all_pops
assert_not_null(ggponc_pops)
len(ggponc_pops), len(top_pop), len(all_pops)

(17530, 39, 17491)

In [20]:
len(ggponc_pops), len(set(ggponc_pops))

(17530, 12005)

In [21]:
ggponc_intv = session.scalars(select(ggponc.Entity.cui).where(ggponc.Entity.type_.in_(['Clinical_Drug', 'Therapeutic', 'Diagnostic']))).fetchall()
assert_not_null(ggponc_intv)

In [22]:
len(ggponc_intv), len(set(ggponc_intv))

(129119, 15852)

In [23]:
top_10(ggponc_intv)

,0,name
C0087111,4329,Therapeutic Procedure
C1522449,2618,Radiation Therapy
C3665472,2329,Chemotherapy
C0184661,1427,Intervention or Procedure
C0543467,1374,Surgical Procedure
C0011900,1192,Diagnosis
C0436307,1014,Chemoradiotherapy
C0030231,938,Palliative Therapy
C0008838,719,Cisplatin
C1699633,691,Positron Emission Tomography and Computed Tomo...


In [24]:
cuis_l1 = app.state.relationship_mapper.get_related_concepts('C0033578', direction='broad2narrow', max_depth=1)

In [25]:
for c in cuis_l1:
    print(c, app.state.concept_parser.umls_parser.get_umls_text(c))

C1335409 Prostate Phyllodes Tumor
C2678479 Prostate Cancer, Hereditary, 12
C0154088 Carcinoma in situ of prostate
C1970192 Prostate Cancer, Hereditary, 10
C3658267 Prostatic Neoplasms, Castration-Resistant
C1335515 Prostate Neuroendocrine Neoplasm
C1864472 PROSTATE CANCER, HEREDITARY, 8
C1837595 PROSTATE CANCER, HEREDITARY, 3
C1836436 Prostate Cancer, Hereditary, 5
C0282612 Prostatic Intraepithelial Neoplasia
C1836005 Prostate Cancer, Hereditary, 6
C5206985 Prostate Soft Tissue Neoplasm
C1837593 Prostate Cancer, Hereditary, 4
C1332353 Atypical Small Acinar Proliferation of the Prostate Gland
C0496923 Neoplasm of uncertain behavior of prostate
C0376358 Prostate cancer
C2677771 Prostate Cancer, Hereditary, 15
C1334615 Malignant Prostate Phyllodes Tumor
C0154009 Benign Prostate Neoplasm
C1853195 Prostate Cancer, Hereditary, 7
C2677773 Prostate Cancer, Hereditary, 11
C4759295 Non-metastatic prostate cancer
C2678047 Prostate Cancer, Hereditary, X-Linked 2
C2677772 Prostate Cancer, Hereditar

In [26]:
cuis_l2 = app.state.relationship_mapper.get_related_concepts('C0376358', direction='broad2narrow', max_depth=1)
for c in cuis_l2:
    print(c, app.state.concept_parser.umls_parser.get_umls_text(c))

C1335514 Prostate Myeloid Sarcoma
C5205938 Prostate Melanoma
C0280280 None
C4721208 Metastatic castration-resistant prostate cancer
C5205936 Extrarenal Rhabdoid Tumor of the Prostate
C4303101 Recurrent malignant neoplasm of prostate
C5231122 Infiltrating duct carcinoma of prostate
C1297952 Malignant tumor involving prostate by direct extension from bladder
C1330959 Primary malignant neoplasm of prostate
C0347001 Metastatic Malignant Neoplasm in the Prostate Gland
C0349672 Prostate Ductal Adenocarcinoma
C1300585 Small cell carcinoma of prostate
C0007112 Prostate Adenocarcinoma
C0279882 None
C1302530 Prostate Squamous Cell Carcinoma
C1334615 Malignant Prostate Phyllodes Tumor
C5205910 Prostate Malignant Solitary Fibrous Tumor
C0238393 Prostate Sarcoma
C1335516 Prostate Non-Hodgkin Lymphoma
C0600139 CARCINOMA OF PROSTATE
C3898877 Hormone-Resistant Prostate Carcinoma
C4302896 Hormone sensitive prostate cancer
C1335512 Prostate Lymphoma
C1328504 Castration-Resistant Prostate Carcinoma
C5205

# PubMed

In [27]:
from integration.orm import pubmed

In [28]:
len(session.scalars(select(pubmed.Trial)).fetchall())

828356

In [29]:
res = (session.scalars(select(pubmed.Trial).where(pubmed.Trial.pm_id == 38828946)).fetchall())

In [30]:
res

[]

In [31]:
pubmed_pops = session.scalars(select(pubmed.UmlsPopulation.cui)).fetchall()

In [32]:
assert_not_null(pubmed_pops)
len(pubmed_pops), len(set(pubmed_pops))

(7326622, 52568)

In [33]:
top_10(pubmed_pops)

,0,name
C0030705,508305,Patient
C0439234,128012,Year
C0001779,123231,Age
C0043210,84758,Woman
C0001675,81191,Adult
C0162574,81072,Advanced Glycation End Product
C0679646,80689,Participant
C0001792,67315,Elderly
C0027361,64430,People
C0008972,61191,Clinical Study


In [34]:
pubmed_intv = session.scalars(select(pubmed.UmlsIntervention.cui)).fetchall()

In [35]:
assert_not_null(pubmed_intv)
len(pubmed_intv), len(set(pubmed_intv))

(5631219, 68738)

In [36]:
top_10(pubmed_intv)

,0,name
C0032042,143367,Placebo
C0184661,59166,Intervention or Procedure
C0009932,55369,Control Group
C0039798,45632,therapeutic aspects
C0087111,43158,Therapeutic Procedure
C1533734,32074,Administration
C0441833,29741,Group
C1522704,29058,Exercise Pain Management
C1522326,28724,Treat
C1705169,27585,Biomaterial Treatment


# ClinicalTrials.gov

In [37]:
from integration.orm import aact

In [38]:
len(session.scalars(select(aact.Trial)).fetchall())

499882

In [39]:
aact_pops = session.scalars(select(aact.MeshCondition.cui)).fetchall()

In [40]:
assert_not_null(aact_pops)
len(aact_pops), len(set(aact_pops))

(3404118, 4466)

In [41]:
top_10(aact_pops)

,0,name
C0027651,93585,NEOPLASM
C0030660,86115,Pathologic Process
C0027653,61792,Neoplasm by Site
C0027765,60721,Nervous System Disorder
C0007222,58069,Cardiovascular Disorder
C0080276,45711,Genitourinary System Disorder
C0035242,45286,Respiratory Tract Diseases
C3714514,44115,Infection
C0027652,42533,Neoplasms by Histologic Type
C0012242,41117,Disorder of digestive system


In [42]:
aact_intv = session.scalars(select(aact.MeshIntervention.cui).where(aact.MeshIntervention.cui != None)).fetchall()

In [43]:
assert_not_null(aact_intv)
len(aact_intv), len(set(aact_intv))

(1543754, 3994)

In [44]:
top_10(aact_intv)

,0,name
C1258063,101662,Physiological Effects of Drugs
C1258062,97111,Molecular Mechanisms of Pharmacological Action
C0003392,51606,Antineoplastic Agent
C0014432,47365,Enzyme inhibitor
C0243049,32439,Peripheral Nervous System Agents
C0021054,24959,Immunologic substance
C0243051,24556,Neurotransmitter Agents
C0003204,24457,Anti-Infective Agent
C0243058,20585,Sensory System Agents
C0007681,18818,Central Nervous System Depressants


# CIViC

In [ ]:
from integration.orm import civic

In [ ]:
len(session.scalars(select(civic.Evidence)).fetchall())

In [ ]:
civic_diseases = session.scalars(select(civic.Evidence.disease_cui).where(civic.Evidence.disease_cui != None)).fetchall()
civic_phenotypes = session.scalars(select(civic.Phenotype.cui).where(civic.Phenotype.cui != None)).fetchall()
civic_pops = civic_diseases + civic_phenotypes

In [ ]:
assert_not_null(civic_pops)
len(civic_pops), len(set(civic_pops))

In [ ]:
top_10(civic_pops)

In [ ]:
civic_intv = session.scalars(select(civic.Therapy.cui).where(civic.Therapy.cui != None)).fetchall()

In [ ]:
assert_not_null(civic_intv)
len(civic_intv), len(set(civic_intv))

In [ ]:
top_10(civic_intv)